### Experiment: Generalization


**Question**: How well does each of the models (CNN, GRU, Transformer) generalize to other datasets?

**Hypothesis**: The models differ in generalizability, ranging from none at all to above-chance performance

**Result**:

How to perform this?
- Take model trained on 100 Hz SAT1
- Test on test-set
- Test on entire set for SAT2 and AR:
`train_data, val_data, test_data = split_data_on_participants(
    dataset, 100, norm_min1_to_1
)`

In [6]:
%load_ext autoreload
%autoreload 2
import netCDF4
import xarray as xr
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT1_STAGES_SPEED
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
# from braindecode.models.eegconformer import EEGConformer
from mne.io import read_info
import gc
import json
import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
set_global_seed(42)

# data_path = Path("../data/sat1/split_stage_data_100hz.nc")
data_path = DATA_PATH / "sat2/split_stage_data_100hz.nc"

dataset = xr.load_dataset(data_path)

In [3]:
# Truncate to SAT1 length
# dataset = dataset.sel(samples=slice(0, 161 - 1))

In [8]:
# data_path_sat2 = Path("../data/sat2/split_stage_data.nc")
data_path_sat2 = DATA_PATH / "sat1/split_stage_data_100hz.nc"
dataset_sat2 = xr.load_dataset(data_path_sat2)

test_data_sat2, _, _ = split_data_on_participants(
    dataset_sat2,
    100,
    normalization_fn=norm_min1_to_1,
    truncate_sample=len(dataset.samples),
)
test_dataset_sat2 = SAT1Dataset(test_data_sat2, labels=SAT1_STAGES_ACCURACY[1:])

# data_path_ar = Path("../data/ar/split_stage_data_new.nc")
# dataset_ar = xr.load_dataset(data_path_ar)

# test_data_ar, _, _ = split_data_on_participants(
#     dataset_ar,
#     100,
#     normalization_fn=norm_min1_to_1,
#     truncate_sample=len(dataset.samples),
# )
# # Subset only labels that exist in SAT1_STAGES_SPEED, these are the stages that exist in both SAT and AR experiments
# test_data_ar = test_data_ar.sel(labels=SAT1_STAGES_SPEED)
# test_data_ar["channels"] = [
#     "P8",
#     "CP2",
#     "P7",
#     "FC1",
#     "FCz",
#     "P4",
#     "T8",
#     "F7",
#     "CP5",
#     "T7",
#     "Fp2",
#     "P3",
#     "O1",
#     "FC2",
#     "FC6",
#     "CPz",
#     "Fp1",
#     "CP1",
#     "C3",
#     "Cz",
#     "F4",
#     "F8",
#     "CP6",
#     "O2",
#     "C4",
#     "F3",
#     "Pz",
#     "AFz",
#     "Fz",
#     "FC5",
# ]
# test_data_ar = test_data_ar.reindex({"channels": dataset.channels.values}, copy=False)
# # Accuracy since model is trained on SAT1_STAGES_ACCURACY
# test_dataset_ar = SAT1Dataset(test_data_ar, labels=SAT1_STAGES_ACCURACY)
# del test_data_ar, test_data_sat2, dataset_ar, dataset_sat2
del test_data_sat2, dataset_sat2
gc.collect()

2190

In [14]:
def test_generalization(
    model_fn, model_kwargs, data, additional_test_data, additional_train_kwargs=None
):
    print(f"Testing model: {model_fn.__name__}")
    train_kwargs = {
        "logs_path": Path("../logs/exp_generalization_datasets/"),
        "additional_info": {
            "model_fn": model_fn.__name__,
            "model_kwargs": str(model_kwargs),
        },
        "additional_name": f"model_fn-{model_fn.__name__}",
        "labels": SAT1_STAGES_ACCURACY[1:],
    }
    if additional_train_kwargs is not None:
        train_kwargs.update(additional_train_kwargs)
    result = k_fold_cross_validate(
        model_fn,
        model_kwargs,
        data,
        k=len(data.participant),
        normalization_fn=norm_min1_to_1,
        train_kwargs=train_kwargs,
        additional_test_data=additional_test_data,
        gen_kwargs={"labels": SAT1_STAGES_ACCURACY[1:]}
    )
    with open(
        train_kwargs["logs_path"] / f"results_{model_fn.__name__}.json", "w"
    ) as f:
        json.dump(result, f, indent=4)
    print_results(result)

#### CNN

In [ ]:
test_generalization(
    SAT1Base,
    {"n_classes": len(dataset.labels)},
    dataset,
    additional_test_data=[test_dataset_sat2, test_dataset_ar],
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

#### GRU

In [15]:
test_generalization(
    SAT1GRU,
    {
        "n_channels": len(dataset.channels),
        "n_samples": len(dataset.samples),
        "n_classes": len(dataset.labels),
    },
    dataset,
    additional_test_data=[test_dataset_sat2],
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

Testing model: SAT1GRU
Fold 1: test fold: ['S1']


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


  0%|          | 0/992 [00:00<?, ? batch/s]

  0%|          | 0/992 [00:00<?, ? batch/s]

  0%|          | 0/992 [00:00<?, ? batch/s]

...

  0%|          | 0/993 [00:00<?, ? batch/s]

  0%|          | 0/993 [00:00<?, ? batch/s]

  0%|          | 0/993 [00:00<?, ? batch/s]

  0%|          | 0/993 [00:00<?, ? batch/s]

Fold 18: Accuracy: 0.8961092715231788
Fold 18: F1-Score: 0.8833736102960204
Test set 0
Accuracies
[0.8963681048607318, 0.8920157901438941, 0.8944662095984329, 0.9007566204287516, 0.8843265503875969, 0.8917220990391722, 0.890295358649789, 0.8978599221789884, 0.8917748917748918, 0.8716783698196338, 0.8728776665215499, 0.9195031055900621, 0.9015049553407561, 0.9178275160431992, 0.9039614281991138, 0.9021055509980859, 0.8809555408095554, 0.8961092715231788]
F1-Scores
[0.8816305901259747, 0.8823204852019609, 0.8818645646583552, 0.888776542025939, 0.8785660261667263, 0.8757531379069816, 0.8816280718862048, 0.8888107541650245, 0.8819860813929163, 0.8575611570941912, 0.860555115655709, 0.9105581034626399, 0.8900492705696866, 0.9049533166144985, 0.8947304548620014, 0.8973976652098492, 0.8667069847566378, 0.8833736102960204]
Average Accuracy: 0.8947838306615213, std: 0.012326850889211148
Average F1-Score: 0.8837345517806288, std: 0.013309624579778006
Test set 1
Accuracies
[0.5363670982482863, 0.

#### Transformer

In [13]:
test_generalization(
    TransformerModel,
    {
        "n_features": len(dataset.channels),
        "n_heads": 10,
        "ff_dim": 512,
        "n_layers": 6,
        "n_samples": len(dataset.samples),
        "n_classes": len(dataset.labels),
    },
    dataset,
    additional_test_data=[test_dataset_sat2, test_dataset_ar],
    additional_train_kwargs={"weight_decay": 0.001, "label_smoothing": 0.0001},
)

Testing model: TransformerModel
Fold 1: test fold: ['S1']


/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

  0%|          | 0/1203 [00:00<?, ? batch/s]

...

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

  0%|          | 0/1205 [00:00<?, ? batch/s]

Fold 18: Accuracy: 0.8673469387755102
Fold 18: F1-Score: 0.8597433716168495
Test set 0
Accuracies
[0.886386668167999, 0.8782791185729276, 0.8604534005037784, 0.8906492699596148, 0.8232287804389218, 0.811399068637376, 0.897362712638429, 0.8560137457044673, 0.8671168831168831, 0.8523619806488333, 0.8087480077917478, 0.8464223385689355, 0.9134142670813685, 0.9196256890142289, 0.7912335226663809, 0.8565434338852348, 0.9010689354275742, 0.8673469387755102]
F1-Scores
[0.8762339007490066, 0.8742333953580277, 0.8510037324305741, 0.8832034387714248, 0.814470388185503, 0.7590760128640459, 0.8988219477363147, 0.8418563708180802, 0.8554171908556129, 0.8455959156275927, 0.8029170816530071, 0.8430289144040138, 0.9100934438445274, 0.910921671762388, 0.733414808319469, 0.8550632377024631, 0.8927993313221654, 0.8597433716168495]
Average Accuracy: 0.8626474867555672, std: 0.0355743844634577
Average F1-Score: 0.8504385641122814, std: 0.04686634977488418
Test set 1
Accuracies
[0.4095799557848194, 0.400049

/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rick/miniconda3/envs/thesis_pytorch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu